# Hugging Face Transformers

## 1. Library Imports

In [ ]:
import json
import numpy as np
import pandas as pd
import torch
import os
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, 
    AutoModelForTokenClassification, 
    Trainer, 
    TrainingArguments,
    DataCollatorForTokenClassification,
    EarlyStoppingCallback
)
from datasets import Dataset as HFDataset
from datasets import DatasetDict
from sklearn.model_selection import train_test_split
from seqeval.metrics import f1_score, precision_score, recall_score, classification_report
import matplotlib.pyplot as plt
import matplotlib.patches as patches

## 3. Constant Definition 

In this cell, I´ll document the type of entities and their correspondant colors.

In [ ]:
# Define entity types and their descriptions
ENTITY_TYPES = {
    "ACTION": "Direct commands or actions mentioned in the message",
    "SITUATION": "Racing context or circumstance descriptions",
    "INCIDENT": "Accidents or on-track events",
    "STRATEGY_INSTRUCTION": "Strategic directives",
    "POSITION_CHANGE": "References to overtakes or positions",
    "PIT_CALL": "Specific calls for pit stops",
    "TRACK_CONDITION": "Mentions of the track's state",
    "TECHNICAL_ISSUE": "Mechanical or car-related problems",
    "WEATHER": "References to weather conditions"
}

# Color scheme for entity visualization
ENTITY_COLORS = {
    "ACTION": "#4e79a7",           # Blue
    "SITUATION": "#f28e2c",         # Orange
    "INCIDENT": "#e15759",          # Red
    "STRATEGY_INSTRUCTION": "#76b7b2", # Teal
    "POSITION_CHANGE": "#59a14f",   # Green
    "PIT_CALL": "#edc949",          # Yellow
    "TRACK_CONDITION": "#af7aa1",   # Purple
    "TECHNICAL_ISSUE": "#ff9da7",   # Pink
    "WEATHER": "#9c755f"            # Brown
}

print("Entity types defined:")
for entity, description in ENTITY_TYPES.items():
    print(f"  - {entity}: {description}")

Types of defined entities:
  - ACTION: Direct commands or actions mentioned in the message
  - SITUATION: Racing context or circumstance descriptions
  - INCIDENT: Accidents or on-track events
  - STRATEGY_INSTRUCTION: Strategic directives
  - POSITION_CHANGE: References to overtakes or positions
  - PIT_CALL: Specific calls for pit stops
  - TRACK_CONDITION: Mentions of the track's state
  - TECHNICAL_ISSUE: Mechanical or car-related problems
  - WEATHER: References to weather conditions


## 4. Load and Explore Data

In [11]:
# Load F1 radio data from JSON file
def load_f1_radio_data(json_file):
    """Load and explore F1 radio data from JSON file"""
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    print(f"Loaded {len(data)} messages from {json_file}")
    
    # Show sample structure
    if len(data) > 0:
        print("\nSample record structure:")
        sample = data[0]
        print(f"  Driver: {sample.get('driver', 'N/A')}")
        print(f"  Radio message: {sample.get('radio_message', 'N/A')[:100]}...")
        
        if 'annotations' in sample and len(sample['annotations']) > 1:
            if isinstance(sample['annotations'][1], dict) and 'entities' in sample['annotations'][1]:
                entities = sample['annotations'][1]['entities']
                print(f"  Number of entities: {len(entities)}")
                if len(entities) > 0:
                    entity = entities[0]
                    entity_text = sample['radio_message'][entity[0]:entity[1]]
                    print(f"  Sample entity: [{entity[0]}, {entity[1]}, '{entity_text}', '{entity[2]}']")
    
    return data



In [12]:
# Load the JSON data
json_file_path = "f1_radio_entity_annotations.json"
f1_data = load_f1_radio_data(json_file_path)

# Count entity types in the dataset
entity_counts = {}
for item in f1_data:
    if 'annotations' in item and len(item['annotations']) > 1:
        if isinstance(item['annotations'][1], dict) and 'entities' in item['annotations'][1]:
            for _, _, entity_type in item['annotations'][1]['entities']:
                entity_counts[entity_type] = entity_counts.get(entity_type, 0) + 1

print("\nEntity type distribution in dataset:")
for entity_type, count in sorted(entity_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  - {entity_type}: {count}")

Loaded 529 messages from f1_radio_entity_annotations.json

Sample record structure:
  Driver: 1
  Radio message: So don't forget Max, use your head please. Are we both doing it or what? You just follow my instruct...
  Number of entities: 3
  Sample entity: [82, 103, 'follow my instruction', 'ACTION']

Entity type distribution in dataset:
  - SITUATION: 255
  - ACTION: 165
  - STRATEGY_INSTRUCTION: 137
  - TECHNICAL_ISSUE: 137
  - WEATHER: 112
  - POSITION_CHANGE: 83
  - INCIDENT: 78
  - TRACK_CONDITION: 62
  - PIT_CALL: 42


## 5. Preprocessing F1 Radio Data

In [27]:
def preprocess_f1_data(data):
    """Extract and preprocess F1 radio data with valid annotations"""
    processed_data = []
    skipped_count = 0
    
    for item in data:
        if 'radio_message' not in item or 'annotations' not in item:
            skipped_count += 1
            continue
            
        text = item['radio_message']
        
        # Skip items with empty or null text
        if not text or text.strip() == "":
            skipped_count += 1
            continue
            
        # Extract entities if they exist in expected format
        if len(item['annotations']) > 1 and isinstance(item['annotations'][1], dict):
            annotations = item['annotations'][1]
            if 'entities' in annotations and annotations['entities']:
                entities = annotations['entities']
                
                # Add to processed data
                processed_data.append({
                    'text': text,
                    'entities': entities,
                    'driver': item.get('driver', None)
                })
            else:
                skipped_count += 1
        else:
            skipped_count += 1
    
    print(f"Processed {len(processed_data)} messages with valid annotations")
    print(f"Skipped {skipped_count} messages with missing or invalid annotations")
    
    # Show a sample of processed data
    if processed_data:
        sample = processed_data[10]
        print("\nSample processed message:")
        print(f"Text: {sample['text']}")
        print("Entities:")
        for start, end, entity_type in sample['entities']:
            entity_text = sample['text'][start:end]
            print(f"  - [{start}, {end}] '{entity_text}' ({entity_type})")
    
    return processed_data



In [28]:
# Preprocess the loaded data
processed_f1_data = preprocess_f1_data(f1_data)

Processed 399 messages with valid annotations
Skipped 130 messages with missing or invalid annotations

Sample processed message:
Text: Max, we've currently got yellows in turn 7. Ferrari in the wall, no? Yes, that's Charles stopped. We are expecting the potential of an aborted start, but just keep to your protocol at the moment.
Entities:
  - [159, 194] 'keep to your protocol at the moment' (ACTION)
  - [5, 42] 'we've currently got yellows in turn 7' (SITUATION)
  - [98, 148] 'We are expecting the potential of an aborted start' (SITUATION)
  - [44, 63] 'Ferrari in the wall' (INCIDENT)
  - [74, 96] 'that's Charles stopped' (INCIDENT)


## 6. Covert to BIO tagging format

Deeper BIO tagging format information can be searched [here](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging)).

### BIO Format Explanation

The **BIO format** is a way to label words in a sentence to indicate if they are part of a named entity, and if so, where in the entity they belong. It uses three types of labels:

- **B- (Beginning)**: The first word in an entity.
- **I- (Inside)**: Any word inside the entity that isn't the first one.
- **O (Outside)**: Words that are not part of any entity.

---

### Example Radio

Here is an example of a radio message from Max Verstappen´s track engineer: 

**Text:**  
*"Max, we've currently got yellows in turn 7. Ferrari in the wall, no? Yes, that's Charles stopped. We are expecting the potential of an aborted start, but just keep to your protocol at the moment."*

Here are the entities mentioned in the message:

1. **'keep to your protocol at the moment'** (ACTION)
2. **'we've currently got yellows in turn 7'** (SITUATION)
3. **'We are expecting the potential of an aborted start'** (SITUATION)
4. **'Ferrari in the wall'** (INCIDENT)
5. **'that's Charles stopped'** (INCIDENT)

---

### Breaking the Sentence

We break the sentence into words and then tag them as follows:

| Word            | BIO Tag          |
|-----------------|------------------|
| Max,            | O                |
| we've           | O                |
| currently       | O                |
| got             | O                |
| yellows         | O                |
| in              | O                |
| turn            | O                |
| 7.              | O                |
| Ferrari         | B-INCIDENT       |
| in              | I-INCIDENT       |
| the             | I-INCIDENT       |
| wall,           | I-INCIDENT       |
| no?             | O                |
| Yes,            | O                |
| that's          | B-INCIDENT       |
| Charles         | I-INCIDENT       |
| stopped.        | I-INCIDENT       |
| We              | B-SITUATION      |
| are             | I-SITUATION      |
| expecting       | I-SITUATION      |
| the             | I-SITUATION      |
| potential       | I-SITUATION      |
| of              | I-SITUATION      |
| an              | I-SITUATION      |
| aborted         | I-SITUATION      |
| start,          | I-SITUATION      |
| but             | O                |
| just            | O                |
| keep            | B-ACTION         |
| to              | I-ACTION         |
| your            | I-ACTION         |
| protocol        | I-ACTION         |
| at              | I-ACTION         |
| the             | I-ACTION         |
| moment.         | I-ACTION         |




In [31]:
def create_ner_tags(text, entities):
    """Convert character-based entity spans to token-based BIO tags"""
    words = text.split()
    tags = ["O"] * len(words)
    char_to_word = {}
    
    # Create mapping from character positions to word indices
    char_idx = 0
    for word_idx, word in enumerate(words):
        # Account for spaces
        if char_idx > 0:
            char_idx += 1  # Space
        
        # Map each character position to its word index
        for char_pos in range(char_idx, char_idx + len(word)):
            char_to_word[char_pos] = word_idx
        
        char_idx += len(word)
    
    # Apply entity tags
    for start_char, end_char, entity_type in entities:
        # Skip invalid spans
        if start_char >= len(text) or end_char > len(text) or start_char >= end_char:
            continue
            
        # Find word indices for start and end characters
        if start_char in char_to_word:
            start_word = char_to_word[start_char]
            # Find the last word of the entity
            end_word = char_to_word.get(end_char - 1, start_word)
            
            # Tag the first word as B-entity
            tags[start_word] = f"B-{entity_type}"
            
            # Tag subsequent words as I-entity
            for word_idx in range(start_word + 1, end_word + 1):
                tags[word_idx] = f"I-{entity_type}"
    
    return words, tags





In [32]:
def convert_to_bio_format(processed_data):
    """Convert processed data to BIO tagging format"""
    bio_data = []
    mapping_errors = 0
    
    for item in processed_data:
        text = item['text']
        entities = item['entities']
        
        # Convert to BIO tags
        words, tags = create_ner_tags(text, entities)
        
        # Check if we mapped any entities
        if all(tag == "O" for tag in tags) and len(entities) > 0:
            mapping_errors += 1
        
        bio_data.append({
            "tokens": words,
            "ner_tags": tags,
            "driver": item.get('driver', None)
        })
    
    print(f"Converted {len(bio_data)} messages to BIO format")
    print(f"Mapping errors: {mapping_errors} (messages where no entities were mapped)")
    
    # Show an example
    if bio_data:
        sample = bio_data[10]
        print("\nSample BIO tagging:")
        print(f"Original text: {' '.join(sample['tokens'])}")
        for token, tag in zip(sample['tokens'], sample['ner_tags']):
            print(f"  {token} -> {tag}")
    
    return bio_data

In [33]:
# Convert processed data to BIO format
bio_data = convert_to_bio_format(processed_f1_data)

Converted 399 messages to BIO format
Mapping errors: 0 (messages where no entities were mapped)

Sample BIO tagging:
Original text: Max, we've currently got yellows in turn 7. Ferrari in the wall, no? Yes, that's Charles stopped. We are expecting the potential of an aborted start, but just keep to your protocol at the moment.
  Max, -> O
  we've -> B-SITUATION
  currently -> I-SITUATION
  got -> I-SITUATION
  yellows -> I-SITUATION
  in -> I-SITUATION
  turn -> I-SITUATION
  7. -> I-SITUATION
  Ferrari -> B-INCIDENT
  in -> I-INCIDENT
  the -> I-INCIDENT
  wall, -> I-INCIDENT
  no? -> O
  Yes, -> O
  that's -> B-INCIDENT
  Charles -> I-INCIDENT
  stopped. -> I-INCIDENT
  We -> B-SITUATION
  are -> I-SITUATION
  expecting -> I-SITUATION
  the -> I-SITUATION
  potential -> I-SITUATION
  of -> I-SITUATION
  an -> I-SITUATION
  aborted -> I-SITUATION
  start, -> I-SITUATION
  but -> O
  just -> O
  keep -> B-ACTION
  to -> I-ACTION
  your -> I-ACTION
  protocol -> I-ACTION
  at -> I-ACTION

### What the Function Does

The function `create_ner_tags` takes the text and entities and converts them into BIO format. It starts by splitting the text into words. 

Then, it maps each word to a tag: "O" for words that are not part of an entity, "B-" for the first word of an entity, and "I-" for subsequent words inside the entity. 

The function also uses the character positions of the entities to determine which words they correspond to. Once the tags are assigned, the function returns the words and their BIO tags, ready for use in training a Named Entity Recognition (NER) model.